# Welcome! 
This notebook presents some of the code that is described in Chapter 5 of Stefan Klikovits's thesis titled "A Domain-Specific Language Approach to Hybrid CPS Modelling".
You can execute the code cells by clicking the little play button above.

In [1]:
# Import of the CREST library
import crestdsl

# A specific subpackage (e.g. the model) 
# can be imported as follows
import crestdsl.model as crest

Most basic resources and entity definition

In [2]:
# use CREST's domain types to specify the domain
watt = crest.Resource(unit="Watt", domain=crest.REAL)  
lumen = crest.Resource(unit="Lumen", domain=crest.INTEGER)

my_lamp = crest.Entity()
my_lamp.in_port = crest.Input(resource=watt, value=100)
my_lamp.out_port = crest.Output(lumen, 0)
my_lamp.on = crest.State()
my_lamp.off = crest.State()
my_lamp.current = my_lamp.off

A very basic entity type

In [3]:
class MyLamp(crest.Entity):
    in_port = crest.Input(resource=watt, value=100)
    out_port = crest.Output(watt, 0)
    on = crest.State()
    off = crest.State()
    current = off
    
my_new_lamp = MyLamp()
my_other_lamp = MyLamp()

An entity with dynamic behaviour

In [4]:
class DynamicLamp(crest.Entity):
    in_port = crest.Input(resource=watt, value=100)
    out_port = crest.Output(watt, 0)
    on = crest.State()
    off = crest.State()
    current = off
    
    off_to_on = crest.Transition(source=off, target=on, \
        guard=(lambda self: self.in_port.value >= 100))
    
    @crest.transition(source=on, target=off)
    def on_to_off(self):
        return self.in_port.value < 100
    
    # output = 90 watt output * 15 lumen per watt
    output_when_on = crest.Update(state=on, target=out_port, \
        function=(lambda self, dt: 90 * 15))
    
    
    @crest.update(state=off, target=out_port)
    def output_when_off(self, dt):
        return 0

Subclassing as form of extension and specialisation

In [5]:
switch = crest.Resource(unit="lampSwitch", domain=["on", "off"])
class SwitchLamp(DynamicLamp):
    switch_input = crest.Input(resource=switch, value="off")
        
    @crest.transition(source="on", target="off")
    def switch_off(self):  # extend DynamicLamp functionality
        return self.switch_input.value == "off"
        
    @crest.transition(source="off", target="on")
    def off_to_on(self):  # override DynamicLamp functionality
        return self.in_port.value >= 100 and \
            self.switch_input.value == "on" 

Parameterisable entities (with `__init__` constructors)

In [6]:
factor = crest.Resource(unit="efficiency", domain=crest.REAL)
class GenericLamp(DynamicLamp):
    threshold = crest.Local(watt, value=100)  # default value
    efficiency = crest.Local(factor, 0.75)
    
    def __init__(self, threshold, efficiency=0.75):
        # constructor: one mandatory + one optional parameter
        self.threshold.value = threshold
        self.efficiency.value = efficiency
    
    @crest.transition(source="off", target="on")
    def off_to_on(self):
        return self.in_port.value >= self.threshold.value
    
    @crest.transition(source="on", target="off")
    def on_to_off(self):
        return self.in_port.value < self.threshold.value
    
    @crest.update(state="on", target="out_port")
    def output_when_on(self, dt):
        return self.in_port.value * self.efficiency.value * 15  
    
powerful_lamp = GenericLamp(300)
efficient_lamp = GenericLamp(50, .97)

Compose entities to a system (with subentities)

In [7]:
class LampComposition(crest.Entity):
    # inputs
    switch_input = crest.Input(resource=switch, value="off")
    in_port = crest.Input(resource=watt, value=100)
    
    # subentities
    big_lamp = GenericLamp(300)
    small_lamp = GenericLamp(100, .9)
    
    # outputs
    big_out = crest.Output(watt, 0)
    small_out = crest.Output(watt, 0)
    
    # states
    on = crest.State()
    off = crest.State()
    current = off
    
    # transitions
    @crest.transition(source=off, target=on)
    def off_to_on(self):
        return self.switch_input.value == "on"
    
    @crest.transition(source=on, target=off)
    def on_to_off(self):
        return self.switch_input.value != "on"
    
    # setting of subentity inputs
    @crest.update(state=on, target=small_lamp.in_port)
    def set_small_lamp_input_when_on(self, dt):
        if self.in_port.value > 100:
            return 100
        else:
            return 0
    
    @crest.update(state=off, target=small_lamp.in_port)
    def set_small_lamp_input_when_off(self, dt):
        return 0
    
    @crest.update(state=on, target=big_lamp.in_port)
    def set_big_lamp_input_when_on(self, dt):
        if self.in_port.value < 100:
            return 0
        else:
            return self.in_port.value - 100
    
    @crest.update(state=off, target=big_lamp.in_port)
    def set_big_lamp_input_when_off(self, dt):
        return 0
    
    
    # connect subentity output to entity output
    @crest.influence(source=big_lamp.out_port, target=big_out)
    def forward_big_output(value):
        # influences only take one parameter: value
        return value
    
    forward_small_output = crest.Influence(
        source=small_lamp.out_port, target=small_out)
        
composition = LampComposition()

## Validator

In [8]:
from crestdsl.model import SystemCheck
# create instance and SystemCheck object
gl = GenericLamp(300, .85)
SystemCheck(gl).check_all() # returns True

gl.current = None # point current state to None

# write to error log: [...] Entity  has no current state
SystemCheck(gl).check_all() # returns False

# write to error log: [...] Entity  has no current state
SystemCheck(gl).check_all(exit_on_error=True) # AssertionError

ERROR:root:Problem in check 'check_current_states': Entity  has no current state
ERROR:root:Problem in check 'check_current_states': Entity  has no current state


AssertionError: Entity  has no current state